In [15]:
# -*- coding: utf-8 -*-
"""
Created on Thu Oct  8 13:53:07 2020

@author: ricks
"""
import pathlib
import xml.etree.ElementTree as ET
from typing import List, Union

import numpy as np
import pandas as pd
import os

In [16]:
folderpath = r"C:\Users\fcstoop01\Desktop\FC Utrecht\Data_analyse\OPTA_" + select_season
matches = os.listdir(folderpath)

NameError: name 'select_season' is not defined

In [17]:
for match in matches:
    f7_filename, f24_filename = import_set_filenames(folderpath, match)

NameError: name 'import_set_filenames' is not defined

In [18]:
def import_set_filenames(folderpath, match):
"""
input:
- folderpath: string that refers to folder containing all matches of a specific season
- match: folder name of 1 of all selected matches.

output:
- filenames for all necesarry files

function description:
This functions opens the right folder for the match and assigns the right filename for:
- OPTA f7 file
- OPTA f24 file
- tracab metadata file
- tracab tracking data file
"""
filepath = folderpath + "\\" + match
for file in os.listdir(filepath):
if file[-5:] == "7.xml":
f7_filename = os.path.join(filepath, file)
elif file[-5:] == "4.xml":
f24_filename = os.path.join(filepath, file)

return f7_filename, f24_filename

IndentationError: expected an indented block (<ipython-input-18-b5355de5a321>, line 16)

In [19]:
def create_playerDB(f7_filename):

    """
    Function that parses an f7 file to a workable pandas dataframe.

    The input is an xml that contains valuable playerinformation.

    Different steps are taken to get to the final database.
    These steps are based on the strange and weird structure of the xml file which was hard to work with.

    1. Create a dataframe of all strating players. This contains:
        - Formation place
        - playerID
        - Position
        - shirt number
        - status (Starter or substitute)
    2. create a dataframe of all player info. This contains:
        - playerID
        - First_name
        - last_name
        - full_name
    3. merge both dataframes
    4. Add playing time and work around substitutions. This contains:
        - full time in minutes
        - minutes seperately
        - seconds seperately
    5. Add teamID for each player (THIS WAS TRICKY, NEED TO CHECK FOR MULTIPLE MATCHES)

    output is dataframe containing the above
    """

    tree = ET.parse(f7_filename)
    root = tree.getroot()

    # match_id = int(root.find("SoccerDocument").get("uID")[1:])

    gameinfo = root.findall("SoccerDocument")[0]
    # gameinfo = gameinfo_1[0]

    formation_place = []
    player_id = []
    position = []
    jersey_no = []
    status = []

    for neighbor in gameinfo.iter("MatchPlayer"):
        formation_place.append(neighbor.get("Formation_Place"))
        player_id.append(neighbor.get("PlayerRef")[1:])
        position.append(neighbor.get("Position"))
        jersey_no.append(neighbor.get("ShirtNumber"))
        status.append(neighbor.get("Status"))

    starting_players = pd.DataFrame(
        {
            "formation_place": formation_place,
            "player_id": player_id,
            "position": position,
            "jersey_no": jersey_no,
            "status": status,
        }
    )

    p_id = []
    first_name = []
    last_name = []
    player_name = []

    for neighbor in gameinfo.iter("Player"):
        p_id.append(neighbor.get("uID")[1:])
        first_name.append(neighbor.find("PersonName").find("First").text)
        last_name.append(neighbor.find("PersonName").find("Last").text)
        player_name.append(first_name[-1] + " " + last_name[-1])

    bench_players = pd.DataFrame(
        {
            "first_name": first_name,
            "player_id": p_id,
            "last_name": last_name,
            "player_name": player_name,
        }
    )

    players = starting_players.merge(bench_players, on="player_id", how="inner")

    time = []
    period_id = []
    player_off = []
    player_on = []

    for neighbor in gameinfo.iter("Substitution"):
        time.append(int(neighbor.get("Min")) + int(neighbor.get("Sec")) / 60)
        period_id.append(neighbor.get("Period"))
        player_off.append(neighbor.get("SubOff")[1:])
        if not neighbor.get("Retired") == '1':
            player_on.append(neighbor.get("SubOn")[1:])
        else:
            player_on.append('None')
    subs = pd.DataFrame(
        {
            "time": time,
            "period_id": period_id,
            "player_off": player_off,
            "player_on": player_on,
        }
    )

    players["start_min"] = 0
    players["end_min"] = 0

    for neighbor in gameinfo.iter("Stat"):
        if neighbor.get("Type") == "match_time":
            match_length = int(neighbor.text)

    players.loc[players["status"] == "Start", "end_min"] = match_length

    for index, content in subs.iterrows():
        players.loc[players["player_id"] == content["player_off"], "end_min"] = content[
            "time"
        ]
        players.loc[
            players["player_id"] == content["player_on"], "start_min"
        ] = content["time"]
        players.loc[
            players["player_id"] == content["player_on"], "end_min"
        ] = match_length

    for neighbor in gameinfo.iter("Booking"):
        if neighbor.get("Card") == "Red":
            players.loc[
                players["player_id"] == neighbor.get("PlayerRef")[1:], "end_min"
            ] = (int(neighbor.get("Min")) + int(neighbor.get("Sec")) / 60)

    players["mins_played"] = players["end_min"] - players["start_min"]

    # players["match_id"] = match_id

    home_away = []

    for team in gameinfo.findall("Team"):
        home_away.append(team.get("uID")[1:])

    players = players[players.mins_played != 0]
    players = players.reset_index(drop=True)

    subs_index = players[(players.status == "Sub")].index
    diff_subs_index = np.diff(subs_index)

    for i in range(len(subs_index) - 1):
        if subs_index[i + 1] - subs_index[i] > 1:
            index_finder = subs_index[i] + 1
            players.loc[:index_finder, "team"] = home_away[0]
            players.loc[index_finder:, "team"] = home_away[1]

    if np.max(diff_subs_index) == 1:
        if np.min(subs_index) < 20:
            index_finder = 11
            players.loc[:index_finder, "team"] = home_away[0]
            players.loc[index_finder:, "team"] = home_away[1]

        else:
            index_finder = 11
            players.loc[:index_finder, "team"] = home_away[0]
            players.loc[index_finder:, "team"] = home_away[1]

    return players, home_away

In [20]:

def parse_f24(file_name):

    # parse the xml and convert to a tree and root
    tree = ET.parse(file_name)
    root = tree.getroot()

    # get the main game info from the single 'Game' node
    gameinfo = root.findall("Game")
    gameinfo = gameinfo[0]
    game_id = gameinfo.get("id")
    home_team_id = gameinfo.get("home_team_id")
    home_team_name = gameinfo.get("home_team_name")
    away_team_id = gameinfo.get("away_team_id")
    away_team_name = gameinfo.get("away_team_name")
    competition_id = gameinfo.get("competition_id")
    competition_name = gameinfo.get("competition_name")
    season_id = gameinfo.get("season_id")

    Edata_columns = [
        "id",
        "event_id",
        "type_id",
        "period_id",
        "min",
        "sec",
        "outcome",
        "player_id",
        "team_id",
        "x",
        "y",
        "sequence_id",
        "possession_id",
    ]

    Q_ids = []
    Q_values = []
    Edata = []

    # loop through each ball node and grab the information
    for i in root.iter("Event"):

        # get the info from the ball node main chunk
        id_ = int(i.get("id"))
        event_id = i.get("event_id")
        type_id = i.get("type_id")
        period_id = int(i.get("period_id"))
        outcome = int(i.get("outcome"))
        min_ = int(i.get("min"))
        sec = int(i.get("sec"))
        player_id = i.get("player_id")
        team_id = i.get("team_id")
        x = i.get("x")
        y = i.get("y")
        possession_id = i.get("possession_id")
        sequence_id = i.get("sequence_id")

        Edata_values = [
            id_,
            event_id,
            type_id,
            period_id,
            min_,
            sec,
            outcome,
            player_id,
            team_id,
            x,
            y,
            sequence_id,
            possession_id,
        ]

        # find all of the Q information for that file
        Qs = i.findall("./Q")

        # create some empty lists to append the results to
        qualifier_id = []
        Q_value = []

        # loop through all of the Qs and grab the info
        for child in Qs:
            qualifier_id.append(child.get("qualifier_id"))
            Q_value.append(child.get("value", default="1"))

        Q_ids.append(qualifier_id)
        Q_values.append(Q_value)
        Edata.append(Edata_values)

    # Stack all ball Data
    df = pd.DataFrame(np.vstack(Edata), columns=Edata_columns)

    unique_Q_ids = np.unique(np.concatenate(Q_ids))

    # create an array for fast assignments
    Qarray = np.zeros((df.shape[0], len(unique_Q_ids)))
    Qarray = Qarray.astype("O")
    Qarray[:] = np.nan

    # dict to relate Q_ids to array indices
    keydict = dict(zip(unique_Q_ids, range(len(unique_Q_ids))))

    # iter through all Q_ids, Q_values, assign values to appropriate indices
    for idx, (i, v) in enumerate(zip(Q_ids, Q_values)):
        Qarray[idx, [keydict.get(q) for q in Q_ids[idx]]] = Q_values[idx]

    # df from array
    Qdf = pd.DataFrame(Qarray, columns=unique_Q_ids, index=df.index)

    # combine
    game_df = pd.concat([df, Qdf], axis=1)

    # assign game values
    game_df["competition_id"] = competition_id
    game_df["game_id"] = game_id
    game_df["home_team_id"] = home_team_id
    game_df["home_team_name"] = home_team_name
    game_df["away_team_id"] = away_team_id
    game_df["away_team_name"] = away_team_name
    game_df["competition_id"] = competition_id
    game_df["competition_name"] = competition_name
    game_df["season_id"] = season_id
    game_df["competition_id"] = competition_id

    game_df[["id", "period_id", "min", "sec", "outcome", "140", "141"]] = game_df[
        ["id", "period_id", "min", "sec", "outcome", "140", "141"]
    ].astype("float")

    game_df["x"] = pd.to_numeric(game_df["x"])
    game_df["y"] = pd.to_numeric(game_df["y"])

    game_df[['x', '140']] = game_df[['x', '140']] / 100 * 105
    game_df[['y', '141']] = game_df[['y', '141']] / 100 * 68
    for i in root.iter("Game"):
        play_date = i.get("game_date").split('T')[0]   

    # write to csv
    return game_df, play_date